In [1]:
%matplotlib inline
import numpy as np
import datetime 
import matplotlib.pyplot as plt
import os
import json
import subprocess

#Do not use the following lines when on lustre, only use them locally
#os.environ['staging_dir'] = '/Users/josaitis/gnuradio_tutorials/ridz_and_scripts/ridz_files/'
#os.environ['which_day'] = '20190917'
#os.environ['DEBUG'] = '0'
#os.environ['DATA_PATH'] = '/Users/josaitis/gnuradio_tutorials/ridz_and_scripts/'
#

#Change these lines based on whether or not you are working locally or on lustre
os.environ['USE_SDR_NO_CONNECTION'] = '1'
os.environ['SDR_NO_CONNECTION'] = 'SDR_nothing_connected_terminated.txt' # #file in DATA_PATH. Only used if USE_SDR_NO_CONNECTION is True. Used in proces() of sp_handling_raw.py, which is lauched in various args of sp_rawview.py
os.environ['SP_RAWVIEW_PATH'] ='/users/ajosaiti/anaconda/envs/py2.7env/rids/scripts/sp_rawview.py'#'/Users/josaitis/anaconda/lib/python2.7/site-packages/EGG-INFO/scripts/sp_rawview.py'# lustre: /users/ajosaiti/anaconda/envs/py2.7env/rids/scripts/sp_rawview.py
os.environ['SDR_NO_CONNECTION_PATH']='/lustre/aoc/projects/hera/ajosaiti/SDR_RFI_monitoring/HERA_daily_RFI/'
#

STR_DAY = os.environ['which_day']
DEBUG = bool(int(os.environ['DEBUG'])) #False
print('Date path: '+str(os.environ['DATA_PATH']))


def execute_sp_rawview_processes():
    #### check that these file ids aren't in processed_fileid.txt
    processed_file_dir = os.path.dirname(os.environ['DATA_PATH'])
    processed_fileid = np.loadtxt(os.path.join(processed_file_dir, 'processed_fileid.txt'), dtype=np.int)
   
    # filter out sessions already processed
    unprocessed_files = []
    pathname = []
    for diritem in os.listdir(os.environ['staging_dir']):
        
        if os.path.isdir(diritem) or str(diritem).isdigit():
            pathname = str(str(os.environ['staging_dir'])+'/'+str(diritem)+'/')
            print('  '+str(diritem)+' considered to be a dir.')
            for dir_file in os.listdir(pathname):
                name=str(dir_file)#file["name"])
                fileid = int(name[int(name.find('.')+1):name.find('-')])
                if (fileid not in processed_fileid) or DEBUG:
                    unprocessed_files.append(name)
                    
        else:
            pathname = str(os.environ['staging_dir']+'/')
            name=str(diritem)#file["name"])

            fileid = int(name[int(name.find('.')+1):name.find('-')])
            if (fileid not in processed_fileid) or DEBUG:
                unprocessed_files.append(name)

    %cd {pathname}
    str_args = str( str( ','.join([str(s) for s in os.listdir(pathname) if s.endswith('.ridz') and STR_DAY in s and os.stat(s).st_size !=0])))
    %run {os.environ['SP_RAWVIEW_PATH']} --wf raw -f '50.,100.' {str_args}
    %run {os.environ['SP_RAWVIEW_PATH']} --wf raw -f '100.,150.' {str_args}
    %run {os.environ['SP_RAWVIEW_PATH']} --wf raw -f '150.,200.' {str_args}
    %run {os.environ['SP_RAWVIEW_PATH']} --wf raw -f '200.,250.' {str_args}
    %run {os.environ['SP_RAWVIEW_PATH']} --totalpower raw -f '50.,250.' {str_args}
    %run {os.environ['SP_RAWVIEW_PATH']} --totalpower raw {str_args}
    
    return pathname 

pathname = execute_sp_rawview_processes()

#create pathname environment variable so I can add librarian events in run_notebook.sh. This pathname may be different than the staging_dir var.
os.environ['staging_pathname']=pathname

Date path: 
[Errno 2] No such file or directory: '[]'
/lustre/aoc/projects/hera/ajosaiti/SDR_RFI_monitoring/HERA_daily_RFI


TypeError: coercing to Unicode: need string or buffer, list found